# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at mlta-2022-spring@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1H5ZONrb2LMOCixLY7D5_5-7LkIaXO6AGEaV2mRdTOMY/edit?usp=sharing)　Kaggle: [Link](https://www.kaggle.com/c/ml2022spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2.5hrs
  

0.80710 0.80960

## Download Dataset

In [1]:
# Download link 1
!gdown --id '1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb' --output hw7_data.zip

# Download Link 2 (if the above link fails)
# !gdown --id '1qwjbRjq481lHsnTrrF4OjKQnxzgoLEFR' --output hw7_data.zip

# Download Link 3 (if the above link fails)
# !gdown --id '1QXuWjNRZH6DscSd6QcRER0cnxmpZvijn' --output hw7_data.zip

!unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb
To: /content/hw7_data.zip
100% 9.57M/9.57M [00:00<00:00, 44.1MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          
Sun Sep 14 02:57:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M.

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [2]:
# 安装与 Py3.12 兼容的版本
!pip install -U "transformers>=4.44,<5" "tokenizers>=0.19.1" accelerate sacremoses peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 28.6 MB/s eta 0:00:00


In [3]:
# You are allowed to change version of transformers or use other toolkits
#!pip install transformers==4.5.0#有transfoemers.AdamW
#报错：版本不兼容

## Import Packages

In [4]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset
#from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast
import torch
from torch.optim import AdamW
from transformers import BertForQuestionAnswering, BertTokenizerFast, get_linear_schedule_with_warmup

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

## Load Model and Tokenizer






In [5]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [6]:
#model = BertForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
#tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")
model = AutoModelForQuestionAnswering.from_pretrained("luhua/chinese_pretrain_mrc_roberta_wwm_ext_large").to(device)
tokenizer = AutoTokenizer.from_pretrained("luhua/chinese_pretrain_mrc_roberta_wwm_ext_large", use_fast=True)
#bert是cls=101 sep=102 pad=0 和roberta不太一样
CLS = tokenizer.cls_token_id
SEP = tokenizer.sep_token_id
PAD = tokenizer.pad_token_id or 0


# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

Some weights of the model checkpoint at luhua/chinese_pretrain_mrc_roberta_wwm_ext_large were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [7]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)
fp16_training = False

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device
# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

## Read Data

- Training set: 31690 QA pairs
- Dev set: 4131  QA pairs
- Test set: 4957  QA pairs

- {train/dev/test}_questions:
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs:
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions

In [8]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [9]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False)

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

从已有的数据集中找到合适的doc_stride:答案token长度p95+安全余量

In [10]:
import math
import numpy as np
import re

def to_int_or_none(x):
    # 已是 None / int / numpy int / float（含 NaN）
    if x is None:
        return None
    if isinstance(x, (int, np.integer)):
        return int(x)
    if isinstance(x, float):
        return None if math.isnan(x) else int(x)

    # 字符串各种情况
    if isinstance(x, str):
        s = x.strip()
        if s == "":
            return None
        # 常见“空值字符串”
        if s.lower() in {"null", "none", "nan", "na"}:
            return None
        # 纯数字或 "144.0" 这类
        try:
            return int(s)
        except ValueError:
            try:
                return int(float(s))
            except ValueError:
                # 最后兜底：提取第一个整数子串（如 " pos=144 "）
                m = re.search(r"-?\d+", s)
                return int(m.group()) if m else None
    # 其它类型一律视为无答案
    return None


In [11]:
import numpy as np
#offsets计算答案的token长度
def char_span_to_token_span_from_offsets(enc, start_char, end_char_exclusive):
  #offsets = enc["offset_mapping"]#获得offsets_mapping
  # 兼容两种返回类型：Encoding / BatchEncoding
  if hasattr(enc, "offsets"):                 # tokenizers.Encoding
    offsets = enc.offsets
  elif hasattr(enc, "encodings") and enc.encodings:   # BatchEncoding 底层
    offsets = enc.encodings[0].offsets
  else:
    offsets = enc["offset_mapping"]         # 字典风格（BatchEncoding）

 #eg.offsets[(0,1),(1,3)......] 这里的（0,1）是二元组是搬开区间[start,end)的意思
  start_tok = end_tok = None

  # 答案start在的token
  #enumerate(offsets)会生成i, (s, e)  标号，元素
  for i, (s, e) in enumerate(offsets):
      if e > start_char:
          start_tok = i
          break
    # 答案end在的token
  for i in range(len(offsets)-1, -1, -1): #左闭右开
      s, e = offsets[i]#等价于（s，e）
      if s < end_char_exclusive:
          end_tok = i
          break
  return start_tok, end_tok
def answer_token_lengths(questions, tokenized_paragraphs, end_is_inclusive=True):
    lengths, miss,no_answer = [], 0,0
    for q in questions:
        pid = q["paragraph_id"]
        enc = tokenized_paragraphs[pid]
        #if "offset_mapping" not in enc:
            #raise ValueError("错误")
        start_char = to_int_or_none(q["answer_start"])
        end_char   = to_int_or_none(q["answer_end"])
        #出现了None，null
        if start_char is None or end_char is None:
            no_answer += 1
            continue
        end_excl = int(end_char) + 1 if end_is_inclusive else int(end_char)

        st_tok, ed_tok = char_span_to_token_span_from_offsets(enc, start_char, end_excl)
        if st_tok is None or ed_tok is None:
            miss += 1
            continue
        lengths.append(int(ed_tok - st_tok + 1))

    return np.array(lengths, dtype=np.int32), miss

def recommend_doc_stride(lengths, Lw=150, margin=12, p=95):
  #推荐的长度是150-（95%答案token长度+margin）
    p95 = int(np.percentile(lengths, p)) if len(lengths) else 20
    overlap = p95 + margin
    stride  = max(1, min(Lw-1, Lw - overlap))
    return int(stride), {"p": p, "p95": p95, "margin": margin, "Lw": Lw}


In [12]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = 150
        lengths,_=answer_token_lengths(questions, tokenized_paragraphs, end_is_inclusive=True)
        ##### TODO: Change value of doc_stride #####
        #两个连续窗口起始位置之间的距离
        re_doc_stride,_=recommend_doc_stride(lengths, Lw=150, margin=12, p=95)
        self.doc_stride = re_doc_stride

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            #将答案固定在窗口中部了
            '''
            mid = (answer_start_token + answer_end_token) // 2
            paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len
            '''
            #将答案放在窗口任何位置--->窗口起始位置随机但是必须包含答案
            total_len = len(tokenized_paragraph)
            start_min = max(0, answer_end_token - self.max_paragraph_len + 1)
            start_max = min(answer_start_token, max(0, total_len - self.max_paragraph_len))
            paragraph_start = random.randint(start_min, start_max)
            paragraph_end = paragraph_start + self.max_paragraph_len
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [CLS] + tokenized_question.ids[:self.max_question_len] + [SEP]
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [SEP]

            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start

            # Pad sequence and obtain inputs to model
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []

            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):

                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [CLS] + tokenized_question.ids[:self.max_question_len] + [SEP]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [SEP]

                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)

                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)

            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [PAD] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        #token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        #全置0
        token_type_ids = [0] * len(input_ids)
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len

        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 32

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [13]:
import numpy as np
import torch

def evaluate(data, output, n_best_size=20, max_answer_len=30, allow_impossible=False):
    """
    data: (input_ids, token_type_ids, attention_mask), 形状 [1, num_windows, seq_len]
    output: 模型输出，含 start_logits / end_logits，形状 [num_windows, seq_len]
    """
    input_ids_all, token_type_ids_all, attn_all = data
    input_ids_all      = input_ids_all[0]      # [num_windows, seq_len]
    token_type_ids_all = token_type_ids_all[0]
    attn_all           = attn_all[0]

    best_answer = ''
    best_score  = float('-inf')
    num_windows = input_ids_all.shape[0]

    for k in range(num_windows):
        ids   = input_ids_all[k]                     # [seq_len]
        attn  = attn_all[k]                          # [seq_len]
        s_log = output.start_logits[k].detach()
        e_log = output.end_logits[k].detach()

        # 有效长度（去掉 PAD 尾部）
        valid_len = int(attn.sum().item())
        ids_v = ids[:valid_len]
        s = s_log[:valid_len].cpu().numpy()
        e = e_log[:valid_len].cpu().numpy()

        # —— 只在 context 区间内找答案 —— #
        tti = token_type_ids_all[k][:valid_len] if token_type_ids_all is not None else None
        if tti is not None and int(tti.max().item()) >= 1:
            # BERT：问题=0，段落=1
            tti_np = tti.cpu().numpy()
            ctx_idx = np.where(tti_np == 1)[0]
            if ctx_idx.size == 0:
                continue
            ctx_start, ctx_end = int(ctx_idx[0]), int(ctx_idx[-1])
        else:
            # RoBERTa：用 [SEP] 定位，[CLS] 问题 [SEP] 段落 [SEP]
            ids_np = ids_v.cpu().numpy()
            sep_pos = np.where(ids_np == SEP)[0].tolist()
            if len(sep_pos) >= 2:
                ctx_start = sep_pos[0] + 1
                ctx_end   = sep_pos[-1] - 1
            else:
                ctx_start, ctx_end = 0, valid_len - 1
        if ctx_start > ctx_end:
            continue

        # —— 屏蔽特殊符号（避免被当作答案） —— #
        neg_inf = -1e30
        s_masked = s.copy()
        e_masked = e.copy()
        special_ids = {CLS, SEP, PAD}
        for i, tid in enumerate(ids_v.cpu().numpy()):
            if tid in special_ids:
                s_masked[i] = neg_inf
                e_masked[i] = neg_inf
        if not allow_impossible and CLS is not None and 0 <= CLS < valid_len:
            s_masked[0] = neg_inf  # 一般 CLS 在位置 0（若模型不同可按需要屏蔽）

        # —— 取 top-n 的起止候选（仅在 context 区间） —— #
        s_ctx = s_masked[ctx_start:ctx_end+1]
        e_ctx = e_masked[ctx_start:ctx_end+1]
        s_top_rel = np.argsort(s_ctx)[-n_best_size:]
        e_top_rel = np.argsort(e_ctx)[-n_best_size:]
        s_cands = (s_top_rel + ctx_start)
        e_cands = (e_top_rel + ctx_start)

        # —— 只枚举合法 span（end ≥ start 且不超长），按 start+end 分数取最大 —— #
        local_best = (None, None, -1e30)
        for si in s_cands:
            max_ej = min(si + max_answer_len - 1, ctx_end)
            for ej in e_cands:
                if ej < si or ej > max_ej:
                    continue
                score = float(s_masked[si] + e_masked[ej])
                if score > local_best[2]:
                    local_best = (si, ej, score)

        si, ej, score = local_best
        if si is None or ej is None:
            continue

        # 解码答案（自动跳过特殊符号；中文再去空格）
        text = tokenizer.decode(ids[si:ej+1], skip_special_tokens=True).replace(' ', '')
        if score > best_score:
            best_score  = score
            best_answer = text

    return best_answer

#def evaluate(data, output):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing
    # Hint: Open your prediction file to see what is wrong
    #对start和end打分 end会出现在start前---->各自寻找编程组队寻找
    '''
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]

    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)

        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob

        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])

    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer.replace(' ','')
    '''


## Training

In [14]:
from transformers import get_scheduler
num_epoch = 1
validation = True
logging_step = 100
learning_rate = 3e-5
#在colab中因为版本原因，直接从pytorch引入AdaW
optimizer = AdamW(model.parameters(), lr=learning_rate)

if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader)

#HF自动调整线性学习率
num_training_steps = num_epoch * len(train_loader)

scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=int(0.1 * num_training_steps),
    num_training_steps=num_training_steps,
)
model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0

    for data in tqdm(train_loader):
        # Load all data into GPU
        data = [i.to(device) for i in data]

        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)

        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss

        if fp16_training:
            accelerator.backward(output.loss)
        else:
            output.loss.backward()

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        step += 1

        ##### TODO: Apply linear learning rate decay #####


        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model"
model.save_pretrained(model_save_dir)

Start Training ...


  0%|          | 0/991 [00:00<?, ?it/s]

Epoch 1 | Step 100 | loss = 0.757, acc = 0.715
Epoch 1 | Step 200 | loss = 0.557, acc = 0.768
Epoch 1 | Step 300 | loss = 0.510, acc = 0.782
Epoch 1 | Step 400 | loss = 0.478, acc = 0.808
Epoch 1 | Step 500 | loss = 0.440, acc = 0.810
Epoch 1 | Step 600 | loss = 0.419, acc = 0.820
Epoch 1 | Step 700 | loss = 0.392, acc = 0.837
Epoch 1 | Step 800 | loss = 0.415, acc = 0.816
Epoch 1 | Step 900 | loss = 0.377, acc = 0.830
Evaluating Dev Set ...


  0%|          | 0/4131 [00:00<?, ?it/s]

Validation | Epoch 1 | acc = 0.803
Saving Model ...


## Testing

In [15]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output))

result_file = "result.csv"
with open(result_file, 'w') as f:
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/4957 [00:00<?, ?it/s]

Completed! Result is in result.csv
